# Convertir archivos excel de SENAMHI a archivos CSV

In [1]:
'''instalar openpyxl para abrir varias hojas de un libro de excel'''
# %pip install openpyxl
# %pip install xlrd

'instalar openpyxl para abrir varias hojas de un libro de excel'

In [2]:
import pandas as pd
import numpy as np
import datetime
from calendar import monthrange

In [3]:
from warnings import simplefilter
simplefilter(action="ignore", category=RuntimeWarning)
simplefilter(action="ignore", category=FutureWarning)

In [4]:
t0 = datetime.datetime.now()

# Convertir de formato obtenido a fomato CSV

Leer los datos del archivo CSV 'estaciones_senamhi'.
Las columnas que nos interesan:
 - path_format contiene las rutas relativas de los archivos,
 - path_csv contiene las rutas relativas para guardar los arvhivos resultantes
 - f contiene el tipo de formato que tiene cada archivo <1 , 2 , 3>

## Formato 1, 2, 3 a formato CSV
Definimos los metodos para las diferentes operaciones y cálculos
Para un dataframe de la forma

| anio = 2003 | | | | | | | | | | | | | |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---|
| DIA | ENE | FEB | MAR | ABR | MAY | JUN | JUL | AGO | SEP | OCT | NOV | DIC | TOTAL |
|1|  1  |  2  |  3  |  4  |     |  2  |  3  |  3  |  5  |  1  |     |     |  x|
|2|  0  |  0  |  0  |  0  |  0  |  9  |  8  |  5  |  3  |     |     |     |  y|
|...| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ...|
|30  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  z|
|31  |  5  |  1  |     |     |  5  |  1  |     |     |  5  |  1  |     |     |  w|

In [5]:
'''devuelve la cadena "2003" convertida en INT, enviadole un DF y el Indice donde se encuentra'''
def es_anio(data, indice):
    '''recuperar el valor del indice y columna'''
    cadena = data.iloc[indice,0]
    '''separa la cadena en base al caracter " " (espacio)'''
    an = cadena.split(' ')
    '''devulve el anio, previamente lo convierte en Entero'''
    return int(an[1])

'''devuelve los datos diarios de cada mes en el anio especificado, traspuestos al formato de entrada'''
def anio_datos (data, indice):
    '''declarar fila(indice) de inicio y fin de los datos del anio'''
    filaini = indice + 2
    filafin = indice + 33
    '''extraer los datos contenidos entre los limites'''
    df1 = data.iloc[filaini:filafin , 1:13]
    '''resetear el indice'''
    df1.reset_index(drop=True, inplace=True)
    '''transponer los datos obtenidos y resetear el indice'''
    df2=df1.T
    df2.reset_index(drop=True, inplace = True)
    return df2

#### La siguiente función devuelve un dataframe de la forma

| anio | mes |  1  |  2  | ... | 30 | 31|
| --- | --- | --- | --- | --- | --- | ---|
| yyyy | m01 |  0  |     | ... |  0 |  1|
| yyyy | m02 |  1  |  0  | ... |    |  2|
| .... | ... | ... | ... | ... | .. | ..|
| yyyy | m11 |  0  |  0  | ... |  0 |  0|
| yyyy | m12 |     |  0  | ... |    |  2|

In [6]:
def anio_comple(data, indice):
    '''crear un df vacio con las columnas anio y mes'''
    am = pd.DataFrame(columns = ['anio', 'mes'])
    '''rellenar las columnas mes y anio'''
    am ['mes'] = np.arange(1, 13)
    am ['anio'] = es_anio(data, indice)
    '''recuperar el df transpuesto'''
    dfa = anio_datos(data, indice)
    df = pd.concat([am, dfa], axis = 1)
    return df

'''recorrer todo el libro para recuperar todos los anios'''
def medida(data):
    i = 0
    '''cantidad total de filas en el df'''
    n = data.shape [0]
    dfall = pd.DataFrame()
    while i<n:
        dfanio = anio_comple(data, i)
        dfall = pd.concat([dfall, dfanio], axis = 0)
        dfall.reset_index(drop = True, inplace = True)
        i += 39
    return dfall

In [7]:

'''funcion que devuelve la fecha de inicio y finalizacion de los datos disponibles de un df'''
def cont_val(df1):
    if min(df1.shape)!=0:
        amin = df1['anio'].min()
        amax = df1['anio'].max()
        mmin = df1[df1['anio']==amin]['mes'].min()
        mmax = df1[df1['anio']==amax]['mes'].max()
        dmax = max(monthrange(amax, mmax))
        fecha1=datetime.date(amin, mmin, 1)
        fecha2=datetime.date(amax, mmax, dmax)
    else:
        fecha1 = pd.NaT
        fecha2 = pd.NaT
    return [fecha1, fecha2]

'''funcion que genera el df de cada medicion'''
def generar_df(d1, d2, df_medicion):
    var_fijas = ["anio","mes"]
    var_melt = df_medicion.columns.difference(pd.Index(var_fijas))
    df_dia= pd.melt(df_medicion,id_vars=var_fijas,value_vars=var_melt, var_name="dia",value_name="valor").sort_values(by=["anio","mes","dia"]).reset_index(drop=True)
    df_dia['dia']=df_dia['dia'].astype(int)
    df_datos = df_dia.sort_values(by=["anio","mes","dia"]).reset_index(drop=True)
    
    '''crear un dataframe vacio con las columnas anio, mes, dia, value'''
    df1 = pd.DataFrame(columns=['anio','mes','dia','valor'])

    '''crear una columna con las fechas dentro el rango'''
    df1['fecha']=pd.date_range(start=d1,end=d2)

    '''asignar las columnas de anio, mes, dia con los valores del rango'''
    df1['anio'] = df1['fecha'].dt.year
    df1['mes'] = df1['fecha'].dt.month
    df1['dia'] = df1['fecha'].dt.day

    '''eliminar la columna fecha'''
    df1.drop(['fecha'], axis=1, inplace=True)
    
    '''crear listas para cada ciclo'''
    rango_anio = list(df1['anio'].unique())
    for x in rango_anio:
        rango_mes = list(df1[df1['anio']==x]['mes'].unique())
        for y in rango_mes:
            rango_dia = list(df1[(df1['anio']==x) & (df1['mes']==y)]['dia'].unique())
            for z in rango_dia:
                '''asignar el subdataframe con la condicion'''
                aux = pd.DataFrame()
                aux = df_datos[(df_datos['anio']==x) & (df_datos['mes']==y) & (df_datos['dia']==z)]
                '''evaluar el tamaño del subdataframe'''
                if min(aux.shape)!=0:
                    '''reiniciar el indice del subdataframe'''
                    aux.reset_index(drop=True, inplace=True)
                    '''obtener el indice de la fila con la fecha requerida y asignar el valor correspondiente'''
                    indice = df1.index[(df1['anio']==x) & (df1['mes']==y) & (df1['dia']==z)]
                    df1.loc[indice, 'valor'] = aux.loc[0, 'valor']
    return df1


'''leer datos desde un archivo con formato 1'''
def leer1(path_f01):
    '''leer los datos desde un xlsx, las primeras 7 hojas'''
    cab_1=['anio', 'mes', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    hoj_1=['CODP1', 'CODP2', 'CODP3', 'CODP4', 'CODP5']
#     hoj_1=['CODP1', 'CODP2', 'CODP3', 'CODP4', 'CODP5', 'CODP6', 'CODP7']
    libro = pd.read_excel(path_f01, sheet_name=hoj_1, skiprows=5, names=cab_1)
    return libro

'''retorna el archivo csv, enviando el libro como datos'''
def asignar1(data):
    '''asignar el contenido del DICTIONARY a dataframes'''
    dfpp=pd.DataFrame(data['CODP1'])
    dftmax=pd.DataFrame(data['CODP3'])
    dftmin=pd.DataFrame(data['CODP2'])
    dftmed=pd.DataFrame(data['CODP4'])
    dfhmed=pd.DataFrame(data['CODP5'])
    
    
    '''llamar a la funcion cont_val para calcular el rango de fechas'''
    rpp = cont_val(dfpp)
    rtmax = cont_val(dftmax)
    rtmin = cont_val(dftmin)
    rtmed = cont_val(dftmed)
    rhmed = cont_val(dfhmed)
    col=['fec_ini', 'fec_fin']
    indic = ['pp', 'tmax', 'tmin', 'tmed', 'hmed']
    df_fecha = pd.DataFrame([rpp, rtmax, rtmin, rtmed, rhmed], columns=col, index = indic)
    fecha_min = min(df_fecha['fec_ini'])
    fecha_max = max(df_fecha['fec_fin'])

    '''llamar a la funcion generar_df para generar los df con la misma fecha de inicio y finalizacion'''
    rpp = generar_df(fecha_min, fecha_max, dfpp)
    rtmax = generar_df(fecha_min, fecha_max, dftmax)
    rtmin = generar_df(fecha_min, fecha_max, dftmin)
    rtmed = generar_df(fecha_min, fecha_max, dftmed)
    rhmed = generar_df(fecha_min, fecha_max, dfhmed)
    
    '''concatenar los resultados parciales'''
    df_estacion1 = pd.concat([rpp['anio'], rpp['mes'], rpp['dia'], rpp['valor'], rtmax['valor'], rtmin['valor'], rtmed['valor'], rhmed['valor']], axis=1)

    '''cambiar los nombres de las columnas'''
    df_estacion1.columns = ['anio','mes','dia','pp','tmax','tmin','tmed','hmed']    
    
    return df_estacion1


'''metodo para cambiar de nombre las columnas(nomcols) de un df(data) que no tiene datos'''
def col_nom(data, nomcols):
    '''si el df esta vacio'''
    if data.shape[0] != 0:
        '''retorna el df actualizado'''
        data.columns = nomcols
        return data
    else:
        '''retorna un df vacio con columnas'''
        data0 = pd.DataFrame(columns = nomcols)
        return data0


'''leer datos de un archivo con formato 2'''
def leer2(path_f02):
    '''leer los datos desde un xlsx, las primeras 7 hojas'''
    hoj_2 = ['Hoja1', 'Hoja2', 'Hoja3', 'Hoja4', 'Hoja5']
    try:
        libro = pd.read_excel(path_f02, header = None, sheet_name = hoj_2, skiprows = 4)
    except ValueError:
        libro = pd.read_excel(path_f02, header = None, sheet_name = 'Hoja1', skiprows = 4)
        
    '''asignar el contenido del DICTIONARY a dataframes'''
    if isinstance (libro, dict):
        dpp=pd.DataFrame(libro['Hoja1'])
        dtmax=pd.DataFrame(libro['Hoja2'])
        dtmin=pd.DataFrame(libro['Hoja3'])
        dtmed=pd.DataFrame(libro['Hoja4'])
        dhmed=pd.DataFrame(libro['Hoja5'])

    '''llamar a la funcion para generar el formato horizontal'''
    if isinstance (libro, dict):
        dfpp = medida(dpp)
        dftmax = medida(dtmax)
        dftmin = medida(dtmin)
        dftmed = medida(dtmed)
        dfhmed = medida(dhmed)
    else:
        dfpp = medida(libro)

    '''cambiar las columnas de los df'''
    cols = ['anio', 'mes', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

    if isinstance (libro, dict):
        dfpp = col_nom(dfpp, cols)
        dftmax = col_nom(dftmax, cols)
        dftmin = col_nom(dftmin, cols)
        dftmed = col_nom(dftmed, cols)
        dfhmed = col_nom(dfhmed, cols)
    else:
        dfvacio = pd.DataFrame()
        dfpp = col_nom(dfpp, cols)
        dftmax = col_nom(dfvacio, cols)
        dftmin = col_nom(dfvacio, cols)
        dftmed = col_nom(dfvacio, cols)
        dfhmed = col_nom(dfvacio, cols)
    
    '''llamar a la funcion cont_val para calcular el rango de fechas'''
    fecpp = cont_val(dfpp)
    fectmax = cont_val(dftmax)
    fectmin = cont_val(dftmin)
    fectmed = cont_val(dftmed)
    fechmed = cont_val(dfhmed)
    col=['fec_ini', 'fec_fin']
    indic = ['pp', 'tmax', 'tmin', 'tmed', 'hmed']
    df_fecha = pd.DataFrame([fecpp, fectmax, fectmin, fectmed, fechmed], columns=col, index = indic)
    fecha_min = min(df_fecha['fec_ini'])
    fecha_max = max(df_fecha['fec_fin'])

    '''llamar a la funcion generar_df para generar los df con la misma fecha de inicio y finalizacion'''
    rpp = generar_df(fecha_min, fecha_max, dfpp)
    rtmax = generar_df(fecha_min, fecha_max, dftmax)
    rtmin = generar_df(fecha_min, fecha_max, dftmin)
    rtmed = generar_df(fecha_min, fecha_max, dftmed)
    rhmed = generar_df(fecha_min, fecha_max, dfhmed)

    '''concatenar los resultados parciales'''
    df_estacion2 = pd.concat([rpp['anio'], rpp['mes'], rpp['dia'], rpp['valor'], rtmax['valor'], rtmin['valor'], rtmed['valor'], rhmed['valor']], axis=1)
    '''cambiar los nombres de las columnas'''
    df_estacion2.columns = ['anio','mes','dia','pp','tmax','tmin','tmed','hmed']
    
    return df_estacion2

'''leer datos de un archivo con formato 3'''
def leer3(path_f03):
    df03 = pd.read_csv(path_f03)
    return df03

def grabar(data, path_f):
    ''''grabar el dataframe resultante en un archivo csv'''
    data.to_csv(path_f, index = False)


### Leer archivo estaciones_senamhi.csv

In [8]:
path_file = '../datos/1.0.Estaciones/estaciones_senamhi.csv'
dflista = pd.read_csv(path_file)

### Realizar operaciones para generar archivos CSV de cada estación

In [9]:
total_f = dflista.shape[0]
dflista['archivo_csv']=''
for i in range(0, total_f):
    '''ruta del archivo'''
    nom_arch0 = dflista['nom_archivo'][i]
    cond = dflista['formato'][i]
    ruta_archivo = '../datos/1.0.formato'+str(cond)+'/'+nom_arch0
    if (cond == 1):
        '''para formato 1'''
        dfarchivo = leer1(ruta_archivo)
        df0_res = asignar1(dfarchivo)
    else:
        if (cond == 2):
            '''para formato 2'''
            df0_res = leer2(ruta_archivo)
        else:
            '''para formato 3'''
            df0_res = leer3(ruta_archivo)
            
    
    '''columna fecha'''                
    df0_res['fecha']=df0_res['anio'].astype(str)+'-'+df0_res['mes'].astype(str)+'-'+df0_res['dia'].astype(str)
    df0_res['fecha']=pd.to_datetime(df0_res['fecha'])
    
    '''reordenar las columnas'''
    df0_res = df0_res.reindex(['fecha','pp','tmax','tmin','tmed','hmed'], axis=1)
    
    '''grabar el resultado en la direccion especificada'''
    ran_fech = '_'+str(df0_res['fecha'].iloc[0].year)+str(df0_res['fecha'].iloc[0].month)+'_'+str(df0_res['fecha'].iloc[-1].year)+str(df0_res['fecha'].iloc[-1].month)
    nombre = str(dflista['id'][i])+'_'+dflista['depto'][i]+'_'+dflista['estacion'][i]+ran_fech+'.csv'
    ruta_guardar = '../datos/1.0.formato_csv/'+nombre
    grabar(df0_res, ruta_guardar)
    
    '''guardar el nombre de los archivos en estaciones_senamhi.csv'''
    #dflista['archivo_csv'][i] = nombre
    dflista.loc[i,'archivo_csv'] = nombre 

    '''Grabar resultados'''
    grabar(dflista, path_file)


### Completar información de estaciones: Challapata y Curahuara
- Para estación de Challapata

In [10]:
'''leer archivo estaciones_senamhi.csv'''
ruta_arch = '../datos/1.0.Estaciones/estaciones_senamhi.csv'
dflista = pd.read_csv(path_file)

consul1 = dflista [dflista ['estacion'] == 'challapata'].index
ruta_chall0 = '../datos/1.0.formato_csv/'+dflista['archivo_csv'].iloc[consul1[0]]
ruta_chall1 = '../datos/1.0.formato_csv/'+dflista['archivo_csv'].iloc[consul1[1]]

'''leer los archivos a concatenar'''
df_challapata0 = pd.read_csv(ruta_chall0)
df_challapata1 = pd.read_csv(ruta_chall1)


'''eliminar las filas del 2011 en df_challapata1'''
indice = df_challapata1[(df_challapata1['fecha'] == '2011-12-31')].index
df_challapata1.drop(range(0,indice[0]+1), inplace = True, axis = 0)

'''concatenar challapata0 y challapata1'''
df_challapata = pd.concat([df_challapata0, df_challapata1])
df_challapata.reset_index(drop = True, inplace = True)

'''guardar'''
nombre_ch = '120_oruro_challapata_19424_20182.csv'
path_f3 = '../datos/1.0.formato_csv/'+nombre_ch
df_challapata.to_csv(path_f3, index = False)


# chall = pd.DataFrame()
# chall = chall.append(dflista.iloc[consul1[0]])
# chall.reset_index(drop = True, inplace = True)
# chall.drop(['codigo','nom_archivo','formato'], inplace = True, axis = 1)
# chall.loc[0,'id']=116
# chall.loc[0,'archivo_csv']=nombre_ch

# dflista = dflista.append(chall)
# dflista.drop([consul1[0], consul1[1]], inplace = True, axis = 0)
# dflista.drop(['nom_archivo','formato'], inplace = True, axis = 1)
# dflista.reset_index(drop = True, inplace = True)
# dflista['id']=dflista['id'].astype(int)
# dflista['altitud']=dflista['altitud'].astype(int)


# path_f4 = '../datos/1.0.Estaciones/estaciones_senamhi_generado.csv'
# dflista.to_csv(path_f4, index = False)


- Para estación de Curahuara

In [11]:
'''leer archivo estaciones_senamhi.csv'''
ruta_arch = '../datos/1.0.Estaciones/estaciones_senamhi.csv'
dflista = pd.read_csv(path_file)

consul2 = dflista [dflista ['estacion'] == 'curahuara'].index
ruta_curah0 = '../datos/1.0.formato_csv/'+dflista['archivo_csv'].iloc[consul2[0]]
ruta_curah1 = '../datos/1.0.formato_csv/'+dflista['archivo_csv'].iloc[consul2[1]]

'''leer los archivos a concatenar'''
df_curahuara0 = pd.read_csv(ruta_curah0)
df_curahuara1 = pd.read_csv(ruta_curah1)


'''Crear un DF vacio para el 2012'''
col = df_curahuara0.columns
df_curahuara2 = pd.DataFrame(columns=col)

'''crear una columna con las fechas dentro el rango'''
df_curahuara2['fecha'] = pd.date_range(start='2012-01-01',end='2012-12-31')
df_curahuara2['fecha'] = pd.to_datetime(df_curahuara2['fecha']).dt.date

df_curahuara = pd.concat([df_curahuara0, df_curahuara2, df_curahuara1])
df_curahuara.reset_index(drop = True, inplace = True)

nombre_cu = '121_oruro_curahuara_19759_20193.csv'
path_f4 = '../datos/1.0.formato_csv/'+nombre_cu
df_curahuara.to_csv(path_f4, index = False)


- Crear "estaciones_senamhi_generado.csv"

In [12]:
chall = pd.DataFrame()
chall = chall.append(dflista.iloc[consul1[0]])
chall.reset_index(drop = True, inplace = True)
chall.drop(['codigo','nom_archivo','formato'], inplace = True, axis = 1)
chall.loc[0,'id']=120
chall.loc[0,'archivo_csv']=nombre_ch

consul2 = dflista [dflista ['estacion'] == 'curahuara'].index
curah = pd.DataFrame()
curah = curah.append(dflista.iloc[consul2[0]])
curah.reset_index(drop = True, inplace = True)
curah.drop(['codigo','nom_archivo','formato'], inplace = True, axis = 1)
curah.loc[0,'id']=121
curah.loc[0,'archivo_csv']=nombre_cu


dflista = dflista.append(chall)

dflista = dflista.append(curah)

dflista.drop([consul1[0], consul1[1],consul2[0], consul2[1]], inplace = True, axis = 0)
dflista.drop(['nom_archivo','formato'], inplace = True, axis = 1)
dflista.reset_index(drop = True, inplace = True)
dflista['id']=dflista['id'].astype(int)
dflista['altitud']=dflista['altitud'].astype(int)
dflista['codigo']=dflista['codigo'].fillna(0)
dflista['codigo']=dflista['codigo'].astype(int)

path_f3 = '../datos/1.0.Estaciones/estaciones_senamhi_generado.csv'
dflista.to_csv(path_f3, index = False)

In [13]:
dflista

,id,estacion,depto,altitud,lon,lat,codigo,archivo_csv
0,100,aeropuerto,oruro,3702,-67.079722,-17.952778,4012,100_oruro_aeropuerto_19431_202012.csv
1,101,andamarca,oruro,3762,-67.506389,-18.771944,4001,101_oruro_andamarca_19757_202010.csv
2,102,antequera,oruro,4057,-66.882900,-18.492900,4080,102_oruro_antequera_20131_20195.csv
3,105,chillca,oruro,4025,-66.813889,-17.836944,4026,105_oruro_chillca_19951_202012.csv
4,106,choquecota,oruro,3867,-67.899000,-18.097000,0,106_oruro_choquecota_19911_20098.csv
5,107,corque,oruro,3758,-67.678611,-18.343889,4066,107_oruro_corque_20128_20181.csv
6,108,cosapa,oruro,3906,-68.706389,-18.177778,4005,108_oruro_cosapa_19758_202010.csv
7,111,huachacalla,oruro,3746,-68.257222,-18.787222,4009,111_oruro_huachacalla_19758_20049.csv
8,112,huayllamarca,oruro,3873,-67.939722,-17.835556,4010,112_oruro_huayllamarca_19904_20214.csv
9,113,quillacas,oruro,3724,-66.959722,-19.230000,0,113_oruro_quillacas_19758_201112.csv


### Datos entre limites d1 <= datos <= d2, revisar los archivos CSV en busca de strings y convertirlos a NaN

In [14]:
'''Se define Periodo de Análisis: f1=Fecha de Inicio y f2=Fecha de Finalización '''
f1=pd.Timestamp(1981, 1, 1)
f2=pd.Timestamp(2020, 12, 31)

def genera_rango (d1, d2, columnas, frontera ):
    vacio = pd.DataFrame(columns = columnas)
    vacio['fecha']=pd.date_range(start=d1,end=d2,closed=frontera)
    return vacio
    
def limite (d1, d2, data):
    data['fecha'] = pd.to_datetime(data['fecha'])
    if (d1 < data['fecha'].iloc[0]):
        datainf = genera_rango(d1, data['fecha'].iloc[0], data.columns, 'left')
        data = datainf.append(data, ignore_index = True)
    if (data['fecha'].iloc[-1] < d2):
        datasup = genera_rango(data['fecha'].iloc[-1], d2, data.columns, 'right')
        data = data.append(datasup, ignore_index = True)
    if (d1 > data['fecha'].iloc[0]):
        indiceinf = data[(data['fecha'] == d1)].index
        data.drop(range(0,indiceinf[0]), inplace = True, axis = 0)
        data.reset_index(drop=True, inplace=True)
    if (data['fecha'].iloc[-1] > d2):
        indicesup = data[(data['fecha'] == d2)].index
        data.drop(range(indicesup[-1]+1,data.shape[0]), inplace = True, axis = 0)
        data.reset_index(drop=True, inplace=True)
    return data

'''metodo para cambiar las cadenas en nan y los numeros en float'''
def cambiar(data):
    tam = data.shape[0]
    for x in range (0, tam):
        for y in range (1, 6):
            try:
                data.iloc[x,y] = float(data.iloc[x,y])
            except ValueError:
                data.iloc[x,y] = np.nan
    return data

In [15]:
path_f3 = '../datos/1.0.Estaciones/estaciones_senamhi_generado.csv'
dflista = pd.read_csv(path_f3)

In [16]:
dflista['archivo_csv_corregido'] = ''

for p in dflista['archivo_csv']:
    pin = '../datos/1.0.formato_csv/'+p
    file = pd.read_csv(pin)
    rfile = limite(f1,f2,file) 
    cfile = cambiar(rfile)
    
    cad=p.split('_')
    pmod='obs_'+cad[0]+'_'+cad[2]+'.csv'
    pout = '../datos/1.0.formato_csv_corregido/'+pmod
    cfile.to_csv(pout, index = False)
    
    indice = dflista[dflista['archivo_csv'] == p].index[0]
    dflista.at[indice, 'archivo_csv_corregido'] = pmod

dflista = dflista.sort_values('id', ascending=True)
dflista.reset_index(drop=True, inplace=True)
dflista.to_csv(path_f3, index = False)

## Para generar dataframe para cada variable meteorológica

In [17]:
def creardf(data, col):
    dat = pd.DataFrame()
    dat[col] = data[col]
    return dat

def guardar(data, nom, path='../datos/1.0.variables/'):
    pth = path + nom
    data.to_csv(pth, index = False)

In [18]:
path_f3 = '../datos/1.0.Estaciones/estaciones_senamhi_generado.csv'
dflista = pd.read_csv(path_f3)

In [19]:
temp = pd.read_csv('../datos/1.0.formato_csv_corregido/obs_100_aeropuerto.csv')

dfpp = creardf(temp,['fecha'])
dftmx = creardf(temp,['fecha'])
dftmn = creardf(temp,['fecha'])
dftmd = creardf(temp,['fecha'])
dfhmd = creardf(temp,['fecha'])
nom_col = ['fecha']

for p in dflista['archivo_csv_corregido']:
    pin = '../datos/1.0.formato_csv_corregido/' + p
    file = pd.read_csv(pin)
    dfpp = pd.concat([dfpp, file['pp']], axis=1)
    dftmx = pd.concat([dftmx, file['tmax']], axis=1)
    dftmn = pd.concat([dftmn, file['tmin']], axis=1)
    dftmd = pd.concat([dftmd, file['tmed']], axis=1)
    dfhmd = pd.concat([dfhmd, file['hmed']], axis=1)
    
    cad = p.split('_')
    nom_col.append(str(cad[1]))
    
dfpp.columns = nom_col
dftmx.columns = nom_col
dftmn.columns = nom_col
dftmd.columns = nom_col
dfhmd.columns = nom_col

guardar(dfpp, 'obs_diario_pp.csv')
guardar(dftmx, 'obs_diario_tmax.csv')
guardar(dftmn, 'obs_diario_tmin.csv')
guardar(dftmd, 'obs_diario_tmed.csv')
guardar(dfhmd, 'obs_diario_hmed.csv')


In [20]:
''' Tiempo de ejecucion'''
t1 = datetime.datetime.now()
t1-t0

datetime.timedelta(seconds=3101, microseconds=290862)

In [21]:
'''mostrar dataframe de precipitacion'''
print(dfpp[['100','101','111','117','120','121']].head(5))

    100  101  111  117   120   121
0   0.0  0.0  0.0  NaN   0.0   7.5
1   0.0  1.5  0.0  NaN   0.0   0.0
2   0.0  0.0  2.3  NaN  12.0   0.0
3   2.8  0.0  1.2  NaN  13.3  14.5
4  23.9  0.8  0.0  NaN   0.0   6.5


In [22]:
'''mostrar dataframe de temperatura máxima'''
print(dftmx[['100','116','117','120','201','203']].head(5))

    100  116  117   120   201   203
0  19.8  NaN  NaN  19.5  17.2  20.2
1  20.0  NaN  NaN  21.0  17.9  21.8
2  19.8  NaN  NaN  23.0  17.8  16.4
3  18.6  NaN  NaN  16.4  14.8  18.0
4  16.8  NaN  NaN  17.0  12.5  17.0


In [23]:
'''mostrar dataframe de temperatura mínima'''
print(dftmn[['100','116','117','120','201','203']].head(5))

   100  116  117  120  201  203
0  6.8  NaN  NaN  5.6  3.0  3.6
1  7.9  NaN  NaN  7.5  2.9  3.0
2  5.1  NaN  NaN  6.5  5.3  4.9
3  5.3  NaN  NaN  8.1  4.1  6.2
4  7.1  NaN  NaN  5.2  4.5  5.3


In [24]:
'''datos de las estaciones'''
print(dflista[['id','estacion','depto','altitud','lon','lat']].head(5))

    id    estacion  depto  altitud        lon        lat
0  100  aeropuerto  oruro     3702 -67.079722 -17.952778
1  101   andamarca  oruro     3762 -67.506389 -18.771944
2  102   antequera  oruro     4057 -66.882900 -18.492900
3  105     chillca  oruro     4025 -66.813889 -17.836944
4  106  choquecota  oruro     3867 -67.899000 -18.097000
